# Projeto 1 - Ciência dos Dados

Nome: Amanda Carmo

Nome: Vitor Miada

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [23]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [24]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\User\Desktop\Insper\2º Semestre\Cdados2\p1_dados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [25]:
dados = pd.read_excel('mibr.xlsx', sheet_name='Treinamento')
dados.head()

,Classificacao,Treinamento
0,2,"@smooyacs @mibr tomar no seu cu, tem quanros t..."
1,3,mibr ta igual o cruzeiro kkkkkkkkkkkkkkkkkkkkk...
2,3,@maycon_design @mibr f brasileirinho eu sou a ...
3,3,"@arthurmelo22_ @mibr @chaosec @envy tem boltz,..."
4,2,@mibr @chaosec @envy piada kkkkkkkkkkkkkkkkkkk...


In [26]:
dados_teste = pd.read_excel('mibr.xlsx', sheet_name='Teste')

#renomeando coluna de teste a fim de diferenciar classificação manual e da máquina.
dados_teste.rename(columns={'Unnamed: 1':'Classificação manual'}, inplace=True) 
dados_teste.head()

,Classificação,Teste
0,1,cs é um bom exemplo do quanto a experiencia in...
1,2,"acho que o time precisa de uma pausa, tentar p..."
2,3,"mano, o que ta acontecendo com a mibr"
3,3,@rafael_loppes1 @mibr @abcfc eu tava acompanha...
4,3,mibr decepcionou hoje :/ pelo menos to feliz p...


___
# Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto a ser analisado, é de um time profissional de "Counter Strike Global Offensive" (CSGO), chamado MIBR. O time na época passa por uma derrota e portanto queremos analisar como seus torcedores agem em relação a isso. Foram considerados relevantes, tweets que mostrem satisfação ou insatisfação sobre a organização e **não** sobre a composição do time.

ESCREVER AQUI

### Limpezas dos tweets:

In [27]:
import re 

def cleanup(text):
    
    #Função para substituir pontos por espaços.    
    punctuation = '[!\-.:?;•,]' #Pontuações que vão ser excluídas.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    
    return text_subbed

#### Assinala-se que as classificações são variáveis qualitativas

In [6]:
dados['Classificacao'] = dados['Classificacao'].astype('category')

In [7]:
dados['Classificacao'].cat.categories = ['P', 'I', 'N']

# Visualizando o resultado
dados['Classificacao'].cat.categories

Index(['P', 'I', 'N'], dtype='object')

In [8]:
dados['Classificacao'].head(10)

0    I
1    N
2    N
3    N
4    I
5    I
6    I
7    N
8    N
9    N
Name: Classificacao, dtype: category
Categories (3, object): [P, I, N]

___
## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [9]:
# P(P), P(I), P(N)
dados.Classificacao.value_counts(True)

I    0.5550
N    0.3075
P    0.1375
Name: Classificacao, dtype: float64

In [32]:
# texto_1 = ' '.join(excel[excel.Categoria=='O'].Treinamento)
# texto_2 = ' '.join(excel[excel.Categoria=='R'].Treinamento)
# texto_3 = ' '.join(excel[excel.Categoria=='P'].Treinamento)
# texto_4 = ' '.join(excel[excel.Categoria=='N'].Treinamento)

# print(f'Probabilidade a priori de ser comentário indiferente: {P_I:0.5g}')
# print(f'Probabilidade a priori de ser comentário negativo: {P_N:0.5g}')
# print(f'Probabilidade a priori de ser comentário positivo: {P_P:0.5g}')

### <font color='#005675'>Funções para frequências</font>

Para obter-se a frequência relativa, conforme visto na aula, precisa-se inicialmente do comando `split()` e em seguida tranformar os dados em uma série do pandas, com `pd.Series()`, tal como foi usado a seguir.

Para uma limpeza adicional, foi adicionado às funções `re.sub(r'http\S+', '', tw_completo)`, o qual remove os http que podem intervir na classificação.

#### <font color='#005675'>Frequência Relativa</font>

In [11]:
def Frequencia_Relativa(cat):
    #tira espaços
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    
    #Extra: remoção dos "http" para não enviezar o classificador.
    remove_http = re.sub(r'http\S+', '', tweet)
    text_clean = cleanup(remove_http)
    
    #split() e pd.Series()
    serie_dados = text_clean.split()
    freq_rel = pd.Series(serie_dados).value_counts(True)
    
    return freq_rel

In [12]:
# P(palabra|P)
Frequencia_Relativa('P').head()


e       0.027061
que     0.025802
a       0.023285
de      0.022026
mibr    0.020768
dtype: float64

In [13]:
# P(palavra|N)
Frequencia_Relativa('N').head()

@mibr    0.030651
mibr     0.028948
que      0.025543
o        0.024691
e        0.023414
dtype: float64

In [14]:
# P(palavra|I)
Frequencia_Relativa('I').head()

o        0.031155
@mibr    0.028775
a        0.024232
mibr     0.023799
que      0.022068
dtype: float64

In [15]:
dados_P = dados[dados.Classificacao == 'P'].copy()
dados_P.head(5)

,Classificacao,Treinamento
10,P,"@vrf_jt @mibr @chaosec @envy assim espero, são..."
13,P,@fafonaves @roque_mn lembra do flamengo? fase ...
14,P,"@mibr @chaosec @envy por favor, chamem o @zorl..."
21,P,@chaosec @mibr @geng @flashpoint @joshnissan @...
39,P,rt @guizaokemen: meu único pedido de aniversár...


In [16]:
dados_I = dados[dados.Classificacao == 'I'].copy()
dados_I.head(5)

,Classificacao,Treinamento
0,I,"@smooyacs @mibr tomar no seu cu, tem quanros t..."
4,I,@mibr @chaosec @envy piada kkkkkkkkkkkkkkkkkkk...
5,I,@eldereti @mibr @chaosec @envy obrigado china
6,I,@eldereti @bida @mibr @flashpoint @chaosec tá ...
12,I,@mibr @chaosec @envy fechem as portas..


In [17]:
dados_N = dados[dados.Classificacao == 'N'].copy()
dados_N.head(5)

,Classificacao,Treinamento
1,N,mibr ta igual o cruzeiro kkkkkkkkkkkkkkkkkkkkk...
2,N,@maycon_design @mibr f brasileirinho eu sou a ...
3,N,"@arthurmelo22_ @mibr @chaosec @envy tem boltz,..."
7,N,@gio_fps mas esperar que a comunidade motive e...
8,N,@gaules o chaos não existia na cenário do mund...


#### <font color='#005675'>Frequência Absoluta</font>

In [18]:
def Frequencia_Absoluta(cat):
    
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    remove_http = re.sub(r'http\S+', '', tweet)
    
    text_clean = cleanup(remove_http)
    serie_dados = text_clean.split()
    freq_abs = pd.Series(serie_dados).value_counts(True)
    
    return freq_abs

In [19]:
Frequencia_Absoluta('P').head()

e       0.027061
que     0.025802
a       0.023285
de      0.022026
mibr    0.020768
dtype: float64

In [20]:
Frequencia_Absoluta('N').head()

@mibr    0.030651
mibr     0.028948
que      0.025543
o        0.024691
e        0.023414
dtype: float64

In [21]:
Frequencia_Absoluta('I').head()

o        0.031155
@mibr    0.028775
a        0.024232
mibr     0.023799
que      0.022068
dtype: float64

In [31]:
t_N = ' '.join(dados[dados.Classificacao=='N'].Treinamento)
t_P = ' '.join(dados[dados.Classificacao=='P'].Treinamento)
t_I = ' '.join(dados[dados.Classificacao=='I'].Treinamento)

TypeError: invalid type comparison

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**